In [4]:
from data_processing import import_mnist
from sklearn.preprocessing import MinMaxScaler

c:\Users\Tardy\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


(1, 28, 28, 1)


In [13]:
data = import_mnist('./data/mnist_images/',1)
data = data/255.0
data

(1, 28, 28, 1)


array([[0.01176471, 0.        , 0.        , 0.01176471, 0.02745098,
        0.01176471, 0.        , 0.01176471, 0.        , 0.04313725,
        0.        , 0.        , 0.01176471, 0.        , 0.        ,
        0.01176471, 0.03137255, 0.        , 0.        , 0.01176471,
        0.        , 0.        , 0.        , 0.00784314, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.00392157, 0.01960784, 0.        , 0.04705882, 0.        ,
        0.0627451 , 0.        , 0.        , 0.01568627, 0.        ,
        0.00784314, 0.03137255, 0.01176471, 0.        , 0.01568627,
        0.03137255, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00784314, 0.        ,
        0.        , 0.        , 0.00392157, 0.00784314, 0.00392157,
        0.04705882, 0.        , 0.03137255, 0.        , 0.        ,
        0.02352941, 0.        , 0.04313725, 0.  